In [1]:
import codecs
import re

In [2]:
def fetch_triples(uni_temps):
    triples = [re.findall('(%x<(?P<row>[-]?\d+),(?P<col>\d+)>)',a) for a in uni_temps]
    for i_i, i in enumerate(triples):
        for i_j, j in enumerate(i):
            k = list(j)
            k[1] = eval(k[1])
            k[2] = eval(k[2])
            triples[i_i][i_j] = k
    return triples

In [3]:
def sent2features(sent, uni_temps, triples, target=False):
    features = []
    for i_row, row in enumerate(sent):
        token_features = []
        for i in range(len(uni_temps)):
            feature = uni_temps[i]
            for to_sub in triples[i]:
                try:
                    content = sent[i_row + to_sub[1]][to_sub[2]]
                except IndexError:
                    content = '$'
                feature = re.sub(to_sub[0], content, feature)
            token_features += [feature]
        features += [token_features]
    return features


def sent2feature_ids(sent, uni_temps, triples, laxicon):
    features = []
    for i_row, row in enumerate(sent):
        token_features = []
        for i in range(len(uni_temps)):
            feature = uni_temps[i]
            for to_sub in triples[i]:
                try:
                    content = sent[i_row + to_sub[1]][to_sub[2]]
                except IndexError:
                    content = '$'
                feature = re.sub(to_sub[0], content, feature)
            try:
                token_features += [str(laxicon[feature])]
            except KeyError:
                continue
        features += [token_features]
    return features



#### Read the template

In [4]:
f=codecs.open("feature.tpl",'r','utf-8')
temps = f.read().strip().split('\n')
uni_temps = []
bi_temps = []
for line in temps:
    if line == '# Unigram':
        a = uni_temps
        continue
    if line == '# Bigram':
        a = bi_temps
        continue
    if line == '':
        continue
    a += [line]
triples = fetch_triples(uni_temps)

## Do not call this chunk if the features are output.

#### Read the Data - for Feature Selecting and Encoding

In [5]:
f=codecs.open("trn.samples","r",'utf-8')
s=f.read()
s_chunks=[[r.strip().split('\t') for r in t.strip().split('\n')] for t in s.strip().split('\n\n')]
del s

In [6]:
feature_dict = {}
for i, chunk in enumerate(s_chunks):
    if not i % 1000:
        print(i)
    features = sent2features(chunk, uni_temps, triples)
    for r in features:
        for rr in r:
            try:
                feature_dict[rr] += 1
            except KeyError:
                feature_dict[rr] = 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


In [7]:
simplified_features = feature_dict.copy()
for r in feature_dict.keys():
    if feature_dict[r] < 5:
        simplified_features.pop(r)
p = 0
for r in simplified_features.keys():
    simplified_features[r] = p
    p += 1
f = codecs.open("simplified_features.dict", "w", "utf-8")
f.write(str(simplified_features))
f.close()

## Call from here if features have been output

#### Read the features

In [5]:
f = codecs.open("simplified_features.dict", "r", "utf-8")
simplified_features = eval(f.read())
f.close()

In [6]:
f=codecs.open("test.samples","r",'utf-8')
s=f.read()
s_chunks=[[r.strip().split('\t') for r in t.strip().split('\n')] for t in s.strip().split('\n\n')]
del s

In [7]:
f = codecs.open("test.features", 'w', 'utf-8')
for i, chunk in enumerate(s_chunks):
    if not i % 1000:
        print(i)
    features = sent2feature_ids(chunk, uni_temps, triples, simplified_features)
    features = ['\t'.join(r) for r in features]
    features = '\n'.join(features)
    f.write(features)
    f.write('\n\n')
f.close()

0
1000


## Forming position-of-predicate and target-label files

In [9]:
f=codecs.open("test.samples","r",'utf-8')
s=f.read()
s_chunks=[[r.strip().split('\t') for r in t.strip().split('\n')] for t in s.strip().split('\n\n')]
del s
f.close()
f=codecs.open("labelID.dict","r",'utf-8')
labeldict = eval(f.read())
f.close()
# f = codecs.open("test.labels", 'w', 'utf-8')
f2 = codecs.open("test.pos", 'w', 'utf-8')
pos = []
for sent in s_chunks:
    t = sent[0][5]
    if t[0] == '0':
        pos += ['0']
    else:
        pos += [t[1:]]
#     for token in sent:
#         lab = token[11]
#         f.write(str(labeldict[lab])+'\n')
#     f.write('\n')
f.close()
f2.write('\n'.join(pos))
f2.close()

In [15]:
len(simplified_features)

159089